In [1]:
import pandas as pd
#import sqlite3 
import urllib.request
import warnings
warnings.filterwarnings('ignore')

changes made 6/1:
- moved the fillna stuff to map_values b/c it gets called before case_matcher
- deleted line about loading in unedited detail codes
- added judges_court as a parameter to case_matcher just for clarity
- dropped "Court Name" column b/c "Court" is there already
- merged just on judge and not both judge and year

In [2]:
#read in courts and judges
court_list = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/courts.csv")
judges = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/judges.csv")
#dictionary for replacing the court id with the court name
court_code = dict(court_list[["Court ID", "Court Name"]].values)
#codes=pd.read_csv("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/edited_codes.csv")

In [19]:
#Replace codes in the dataframe with their actual meaning
#Ex. replace "W" with "White" for defendant race
def map_values(df):
    #Fill in NaN values
    df['Race'] = df['Race'].fillna("U") #U is the code for unknowns 
    df['Sentence Y'] = df['Sentence Y'].fillna(0)
    df['Sentence M'] = df['Sentence M'].fillna(0)
    df['Sentence D'] = df['Sentence D'].fillna(0)
    df['Probation D'] = df['Probation D'].fillna(0)
    df['Probation Y'] = df['Probation Y'].fillna(0)
    df['Probation M'] = df['Probation M'].fillna(0)
    df["Charge Class"] = df['Charge Class'].fillna("Un")
    df['Judge'] = df['Judge'].fillna("N/A") #Nans are filled with unknowns 
    
    #f = open("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/edited_codes.csv")
    f = open(r"C:\Users\chris\Documents\GitHub\State-Sentencing-Project\CSV Processing/edited_codes.csv")
    for line in f:
        line = line.strip('\n')
        line = line.split(",")
        df = df.replace({line[2]:{line[0]:line[1]}})
    f.close()
    
    return df

In [35]:
def case_matcher(cases, judges_court):
    """
    cases: a dataframe of cases for 1 specifc court
    judges_court: list of judges for specific court
    """

    cases["Year"]=cases["Last Hearing Date"].str[-4:]

    
    
    #Merges the columns by the judge and the hearing year
    cases['Year']=cases['Year'].astype(int)
    cases['Judge']=cases['Judge'].astype(str)
    judges_court['Year']=judges_court['Year'].astype(int)
    judges_court['Judge']=judges_court['Judge'].astype(str)

    cases = cases.merge(judges_court, how="left",on = ["Judge"])

    #dropping the name and second court name column
    cases=cases.drop(['Name','Court Name'], axis=1)
    
    #Checking merge - can remove when submitting
    unknowns = cases["Judge Full Name"].isna().mean()
    if unknowns > 0.1:
        print(cases["Court"].iloc[1],":", (cases["Judge"]=="N/A").mean(), unknowns)
    
    #Saves the dataframe to a csv
#     file_name = '/Users/hinaljajal/Downloads/{}.csv'.format(court)
#     cases.to_csv(file_name, index = False)
    
    return cases

In [36]:
separate_court_data = []
for court in court_list["Court ID"]:
    try:
        #data = pd.read_csv("/Users/hinaljajal/Downloads/finished_courts/{}.csv".format(court))
        data = pd.read_csv("C:/Users/chris/Documents/case data/finished courts 5.29.4.30/{}.csv".format(court))
    except FileNotFoundError:
        continue
    data = map_values(data)
    #Slices the judges for the particular court 
    judges_court = judges[judges["Court Name"]==court_code[court]]
    #replace court code with name, ie "001C" with "Accomack"
    data["Court"] = court_code[court]
    data = case_matcher(data, judges_court)
    separate_court_data.append(data) #add to list for concatenation
    
final_cases = pd.concat(separate_court_data, ignore_index = True)

<ipython-input-35-7c1d7c11823e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judges_court['Year']=judges_court['Year'].astype(int)
<ipython-input-35-7c1d7c11823e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judges_court['Judge']=judges_court['Judge'].astype(str)


Accomack : 0.0 0.004683032866290197
Albemarle : 0.005561440677966102 0.04399339184826473
Alleghany : 0.001298884653395454 0.0554567273754059
Amelia : 0.03623560855263158 0.08871299342105263
Amherst : 0.005479159760563005 0.005646811380496782
Appomattox : 0.0012760777949375704 0.005150119100594194
Augusta : 0.18913211089135462 0.347507626294794
Bath : 0.0023346303501945525 1.0
Bedford : 0.06896498865657 0.08355815807223006
Bland : 0.007215366242038217 0.008509156050955414
Botetourt : 0.0019395836297244748 0.006108028643239911
Bristol : 0.019025743617608074 0.0310948652720492
Brunswick : 0.008762855230329216 0.06364023611026592
Buchanan : 0.001129385190936684 0.0039454180025485465
Buckingham : 0.1813912333242581 0.26585897086849986
Buena Vista : 0.04602550580113146 0.21958001725956466
Campbell : 0.522095540482032 0.522239861451869
Caroline : 0.0006448265638897124 0.006618737029350726
Carroll : 4.052812651980475e-05 0.018017003800637643
Charles City : 0.5366619115549215 0.5378031383737518

C:\Users\chris\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Danville : 0.09841922947229309 0.09873259057668661
Dickenson : 0.010353168843734882 0.08392839864537978
Dinwiddie : 0.0009903049972953856 0.017217991927765987
Essex : 0.9996096799375488 1.0
Fauquier : 0.007165983551421986 0.05051166378173795
Floyd : 0.9530218384966989 0.9659725749111224
Fluvanna : 0.00022393073076061804 0.007496347796414976
Franklin : 0.0002532097669670534 0.014163754754346546
Frederick : 0.00027691516506121803 0.0861799552979805
Fredericksburg : 0.8655533540900084 0.8764223039909426
Giles : 0.03518633130751514 0.03575450736092369
Gloucester : 0.00491943533437901 0.1702314748799848
Goochland : 0.9989146164978292 1.0
Grayson : 0.9997834091401343 1.0
Greene : 0.999731255038968 1.0
Greensville : 0.005437763838870471 0.15175654081900355
Halifax : 5.140113445327334e-05 0.017443126156525525
Hampton : 9.81789544707284e-05 0.001673220053852201
Hanover : 0.003736647967417198 0.13979289940828402
Henry : 0.003727847607151222 0.5491841673823094
Highland : 0.0002620545073375262 0.0

In [61]:
final_cases["Judge Full Name"].isna()

,Case Number,Court,Last Hearing Date,Charge,Charge Code,Charge Class,Charge Code Section,Concluded By,Sentence Y,Sentence M,Sentence D,Probation Type,Probation Y,Probation M,Probation D,Race,Gender,Year_x,Judge Full Name,Year_y
Judge,,,,,,,,,,,,,,,,,,,,
AVW,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0
CMS,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708,1708
COA,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0
DAM,104,104,104,104,104,104,104,104,104,104,104,104,104,104,104,104,104,104,0,0
DBC,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0
DV,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
DVS,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0
DVW,27379,27379,27379,27379,27342,27379,26499,27379,27379,27379,27379,27379,27379,27379,27379,27379,26789,27379,0,0
DVWM,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0


Outlier courts:
- Madison - Uses first 4 letters of last name instead of judge initials (possibly? not certain)
- Pulaski and Surry - Majority uses ^ but also some cases have initials
- Bath - Last name, also WETS = John E. Wetsel?
- Buckingham - uses both
- Buena Vista - look into same name multiple initials
- Gloucester - just a mess with typos/judges outside of court
- Greensville - uses last name, initials, first letter of last name
- Hanover - ID is 1, 2, 3, instead of names